# **Регуляризация**

✍ В предыдущем юните мы говорили о том, что полиномиальная регрессия склонна к переобучению. Это связано со сложностью модели и её способностью подстраиваться под очень сложные зависимости, из-за которых возникает высокий разброс.

Рассмотрим пример ↓

Обучим модель полиномиальной регрессии третьей степени. Будем использовать данные о жилье в Бостоне и возьмём следующие четыре признака: LSTAT, CRIM, PTRATIO и RM.

Для оценки качества модели будем использовать кросс-валидацию и сравнивать среднее значение метрики на тренировочных и валидационных фолдах. Кросс-валидацию организуем с помощью функции **cross_validate** из модуля **model_selection**:

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
from sklearn import datasets # для импорта данных

# загружаем датасет
boston = datasets.load_boston()
boston_data = pd.DataFrame(
    data=boston.data, #данные
    columns=boston.feature_names #наименования столбцов
)
boston_data['PRICE'] = boston.target

c:\Users\Roman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the

В качестве метрики используем среднюю абсолютную процентную ошибку — MAPE.

In [11]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]
 
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
 
# создаём модель линейной регрессии
lr = linear_model.LinearRegression()
 
# оцениваем качество модели на кросс-валидации, метрика — MAPE
cv_results = cross_validate(lr, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))	

MAPE на тренировочных фолдах: 12.64 %
MAPE на валидационных фолдах: 24.16 %


Что мы видим? Даже при, казалось бы, небольшой, третьей степени полинома мы получили переобучение: на тренировочной выборке 12, а вот на тестовой — 24. Показатели качества отличаются практически в два раза, что говорит о высоком разбросе модели. Ещё более удручающий результат мы получим, если воспользуемся полиномом большей степени (при желании вы можете проверить это самостоятельно).

Как с этим справиться, мы тоже уже знаем.

* Можно попробовать понизить сложность модели (**снизить степень полинома**). Но до какой степени? Можно постепенно перебирать степень полинома до тех пор, пока не получим адекватные результаты, но, согласитесь, процедура не очень приятная.
* Можно воспользоваться **методами регуляризации**.

О втором способе как раз и поговорим подробнее с математической точки зрения.

Для начала вспомним, что такое регуляризация.

***
**Регуляризация** — это способ уменьшения переобучения моделей машинного обучения путём намеренного **увеличения смещения** модели **для уменьшения её разброса**.
***

Регуляризация для линейной регрессии преследует сразу несколько целей. Однако далее мы увидим, что все эти цели на самом деле взаимосвязаны:

* предотвратить переобучение модели;
* включить в функцию потерь штраф за переобучение;
* обеспечить существование обратной матрицы (A.T@A)^-1;
* не допустить огромных коэффициентов модели.

Мы знаем, что большие значения весов — прямое свидетельство переобучения модели линейной регрессии и её нестабильности. Идея регуляризации состоит в наложении ограничения на вектор весов (часто говорят — наложение штрафа за высокие веса). В качестве штрафа принято использовать **норму вектора весов**.

Давайте запишем это на языке линейной алгебры. Вот задача минимизации длины вектора ошибок, о которой мы говорили, когда выводили формулу МНК:

![](data/71.PNG)

*Примечание. Обратите внимание на сумму под знаком корня. У нас она начинается с i = 0. Однако иногда в литературе, например [здесь](https://dyakonov.org/2019/10/31/линейная-регрессия/), можно встретить i = 1, то есть свободный член w0 рекомендуется не регуляризировать (не ограничивать). На самом деле это утверждение эвристическое и может как выполняться, так и нет, в зависимости от особенностей реализации. Мы будем придерживаться реализации в sklearn, в которой **w0 всё-таки включается в регуляризацию**.* 

Порядок нормы ***p*** в общем случае может быть любой — главное, чтобы она была больше 1. Однако на практике распространены только первая и вторая степени, так называемые **L1- и L2-регуляризации**. О них мы поговорим ниже, а пока доведём решение задачи до конца и получим общую формулу.

Поставленная задача оптимизации называется **условной** — мы ищем минимум при некотором условии. Мы ещё не умеем решать такие задачи, но обязательно научимся, а пока на минутку заглянем в теорию оптимизации и поговорим о **методе множителей Лагранжа**. Прелесть данного метода в том, что он позволяет свести условную задачу оптимизации к безусловной, то есть благодаря Лагранжу мы можем перейти от системы к одному уравнению.
***
Метод множителей Лагранжа говорит, что записанная система с ограничением эквивалентна следующей записи:

![](data/72.PNG)

где L(w,a) — функция Лагранжа, которая зависит не только от вектора весов модели w, но и от некоторой константы a>=0 — множителя Лагранжа.

Это и есть финальный результат, на котором мы пока что остановимся. По сути, ничего особо не изменилось по сравнению с изначальной задачей оптимизации. Добавилось только одно слагаемое — ![](data/73.PNG). Заметим, что если a=0, то мы получаем исходную задачу ![](data/74.PNG). Далее мы увидим, что это маленькое слагаемое очень сильно поможет нам победить переобучение модели.

В машинном обучении множитель Лагранжа  принято называть **коэффициентом регуляризации**. Он отвечает за «силу» регуляризации. Чем он больше, тем меньшие значения может принимать слагаемое ![](data/75.PNG), то есть тем сильнее ограничения на норму весов. В этом и была наша цель — ограничить веса.

Теперь разберёмся с **частными случаями**.

***
## **L2-РЕГУЛЯРИЗАЦИЯ**

Начнём мы, как ни странно, с L2-регуляризации, так как она очень наглядно показывает, как регуляризация обеспечивает невырожденность матрицы A.T@A.
***
* **L2-регуляризация (Ridge), или регуляризация по Тихонову** — это регуляризация, в которой порядок нормы p = 2. 
***
Тогда, если подставить p = 2 в наши формулы, то оптимизационная задача в случае L2-регуляризации будет иметь вид:

![](data/76.PNG)

Примечание. Видно, что норма порядка p = 2 на самом деле является знакомой нам длиной вектора. То есть в случае L2-регуляризации мы накладываем ограничение на длину вектора весов w.

В терминах функции Лагранжа задача будет выглядеть как:

![](data/77.PNG)

Примечание. В других реализациях аналитического решения регуляризации Тихонова, отличных от sklearn, где коэффициент w0 не участвует в регуляризации, единичная матрица E заменяется на матрицу I, в которой первый столбец и первая строка — нулевые. Это делается для того, чтобы исключить коэффициент w0 из регуляризации:

![](data/78.PNG)

Что мы в итоге получаем? Преимущество этой формулы в том, что, если a>0, то матрица A.T@A + aE гарантированно является невырожденной, даже если матрица A.T@A таковой не является. Так получается за счёт того, что по диагонали матрицы A.T@A мы добавляем поправки, которые создают линейную независимость между столбцами матрицы.

Продемонстрируем это на примере ↓

### **Пример № 1**

Построить линейную регрессию с L2-регуляризацией, если:

![](https://lms.skillfactory.ru/assets/courseware/v1/1177f21b44348a02e959c76c94516ee7/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_5.png)

Коэффициент регуляризации a = 5.

Давайте составим матрицу наблюдений A для нашей задачи:

![](https://lms.skillfactory.ru/assets/courseware/v1/42ca4fda79a56c1f85f90c77f0fd16ce/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_6.png)

Найдём матрицу Грама A.T@A: 

![](https://lms.skillfactory.ru/assets/courseware/v1/12c5a0f21c020a58afe20e8ac9782386/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_7.png)

Очевидно, что матрица A.T@A вырождена: её второй и третий столбцы являются пропорциональными с коэффициентом 2. Значит, наша классическая формула МНК (без сингулярного разложения) не сработает.

![](data/79.PNG)

Проверим:

In [12]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии по МНК
w_hat = np.linalg.inv(A.T@A)@A.T@y
print(w_hat) 

LinAlgError: Singular matrix

Мы ожидаемо получили ошибку, говорящую о том, что матрица A вырождена. 

Теперь попробуем воспользоваться регуляризацией Тихонова. Для этого составляем матрицу E. Она будет размером 3x3 (количество параметров — 3):

![](https://lms.skillfactory.ru/assets/courseware/v1/3270bf11f0dcaf6b11a50376d7d6fc21/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_8.png)

Добавим регуляризационное слагаемое к матрице A.T@A:

![](https://lms.skillfactory.ru/assets/courseware/v1/7ad2d546088b44fdc84f8eeded7a402a/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_9.png)

Видно, что матрица A.T@A+aE уже не будет вырожденной: её столбцы уже не являются линейно зависимыми, а значит решение будет существовать.

Попробуем найти вектор оценок весов w_ridge по формуле:

In [13]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# единичная матрица
E = np.eye(3)
# коэффициент регуляризации 
alpha = 5
# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
w_hat_ridge = np.linalg.inv(A.T@A+alpha*E)@A.T@y
print(w_hat_ridge) 

[0.6122449  0.29387755 0.5877551 ]


Работает! Мы получили вектор весов.

Итак, мы посмотрели, как работает аналитическое решение L2-регуляризации. Однако в реализации sklearn для решения этой задачи поддерживается сразу несколько методов — как численных (координатный спуск, градиентный спуск или [LBFGS](https://ru.wikipedia.org/wiki/Алгоритм_Бройдена_—_Флетчера_—_Гольдфарба_—_Шанно)), так и аналитических (классическая регуляризация Тихонова или она же через SVD-разложение). По умолчанию метод выбирается автоматически. На простых данных все методы будут показывать примерно одинаковые результаты при одном и том же значении коэффициента регуляризации, однако на реальных данных, когда данные не стандартизированы и присутствует сильная мультиколлинеарность между факторами, результат работы каждого из методов решения задачи оптимизации может значительно отличаться. Имейте это в виду при построении модели. Подробнее о методах вы можете прочитать в документации.

Напомним, что за реализацию линейной регрессии в sklearn отвечает класс [**Ridge**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html). Основной параметр модели, на который стоит обратить внимание — **alpha**, коэффициент регуляризации из формулы Тихонова.

Давайте обучим модель для решения нашей последней задачи, а затем проверим коэффициенты регрессии. Так как мы заранее заложили в матрицу A столбец из единиц, то, чтобы получить корректное решение, параметр **fit_intercept** следует установить в значение False.

In [14]:
from sklearn.linear_model import Ridge

In [15]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
ridge = Ridge(alpha=5, fit_intercept=False)
ridge.fit(A, y)
print(ridge.coef_) 

[0.6122449  0.29387755 0.5877551 ]


Получили тот же самый результат, что и раньше.
***
Наконец, посмотрим, как регуляризация поможет побороть переобучение модели полиномиальной регрессии на наборе данных о домах в Бостоне. Используем те же самые признаки: LSTAT, CRIM, PTRATIO и RM. 

Сразу отметим, что для успешной сходимости численных методов оптимизации, которые используются для решения задачи условной оптимизации, необходима **стандартизация** (нормализация) исходных данных, которая не требовалась для аналитического МНК в классической линейной регрессии (LinearRegression).

*Примечание. Здесь под стандартизацией мы понимаем именно приведение распределения признака к нулевому среднему и единичному стандартному отклонению (StandartScaler), а не стандартизацию векторов, о которой мы говорили в этом модуле. Последнюю также можно использовать в качестве способа масштабирования данных, однако её реализации нет в sklearn.*

Воспользуемся моделью полиномиальной регрессии третьей степени с регуляризацией Тихонова (коэффициент регуляризации возьмём равным 20) и проверим её качество на кросс-валидации по метрике MAPE.

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]
# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
# создаём модель линейной регрессии c L2-регуляризацией
ridge = Ridge(alpha=20, solver='svd')
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(ridge, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 12.54 %
MAPE на валидационных фолдах: 17.02 %


Нам удалось уменьшить ошибку (MAPE) на валидационных фолдах кросс-валидации с 24.16% до 17.02% и сократить разницу в метриках, тем самым уменьшив разброс ответов модели.

Теперь перейдём к L1-регуляризации.
***

### **ЗАДАЧА**

Вычислите коэффициенты линейной регрессии с L2-регуляризацией, используя аналитическую формулу Тихонова, если:

![](https://lms.skillfactory.ru/assets/courseware/v1/26955960b6372607132b2a005abafbc9/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_10.png)

Коэффициент регуляризации a=1.

В качестве ответа приведите значения полученных коэффициентов линейной регрессии, округлив их до второго знака после точки-разделителя.

In [29]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [5, 9, 4, 3, 5],
    [15, 18, 18, 19, 19],
    [7, 6, 7, 7, 7]
]).T
# вектор целевого признака
y = np.array([24, 22, 35, 33, 36])
alpha = 1
E = np.eye(4)
# получаем оценку коэффициентов регрессии по МНК с регуляризацией Тихонова
w_hat_ridge = np.linalg.inv(A.T@A+alpha*E)@A.T@y
print(w_hat_ridge.round(2)) 

[-0.09 -1.71  1.91  0.73]


***
## **L1-РЕГУЛЯРИЗАЦИЯ**

* **L1-регуляризацией, Lasso** (Least Absolute Shrinkage and Selection Operator), называется регуляризация, в которой порядок нормы p = 1.
***

Тогда оптимизационная задача в случае L1-регуляризации будет иметь вид:

![](data/80.PNG)

Примечание. Таким образом, в случае L1-регуляризации мы ограничиваем **сумму модулей весов модели**. Такая величина называется **[нормой Манхэттена](https://ru.wikipedia.org/wiki/Расстояние_городских_кварталов) (расстоянием городских кварталов)**.

Запишем полученную систему в терминах метода Лагранжа:

![](data/81.PNG)

Можно показать, что данная задача имеет аналитическое решение, однако в реализации sklearn оно даже не заявлено как возможное для использования в связи с нестабильностью взятия производной от функции модуля, поэтому мы не будем его рассматривать. Ознакомиться с ним вы можете [здесь](http://www.machinelearning.ru/wiki/images/7/7e/VetrovSem11_LARS.pdf).

В sklearn L1-регуляризация реализована в классе **Lasso**, а заданная выше оптимизационная задача решается алгоритмом **координатного спуска (Coordinate Descent)**.

Давайте посмотрим, как работает Lasso на «игрушечном» примере, а затем применим его для набора данных о домах в Бостоне.

### **Пример № 2**

Построить линейную регрессию с L1-регуляризацией, если:

![](https://lms.skillfactory.ru/assets/courseware/v1/d6b4f37e6250dd672fe066b8642ee778/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_13.png)

Коэффициент регуляризации a=0.1.

Составим матрицу наблюдений A для нашей задачи:

![](https://lms.skillfactory.ru/assets/courseware/v1/e054cc4c29a1b089b542f008c21ff2b1/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_14.png)

Из примера №1 мы уже знаем, что матрица Грама A.T@A будет вырождена, а значит классического МНК-решения (не беря в расчёт сингулярное разложение) не получится.

Попробуем найти коэффициенты регрессии с помощью L1-регуляризации. Для этого подадим нашу матрицу наблюдений A и вектор целевого признака y в модель Lasso.

In [37]:
from sklearn.linear_model import Lasso

In [38]:
# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии с помощью L1-регуляризации
lasso = Lasso(alpha=0.1, fit_intercept=False)
lasso.fit(A, y)
print(lasso.coef_)

[1.14925373 0.         0.71921642]


Вот наша оценка вектора весов:

![](data/82.PNG)

Сразу обращаем внимание, что, в отличие от регуляризации Тихонова, L1-регуляризация «занулила» коэффициент, стоящий при факторе x1. Это произошло не случайно, так как это особенность данного метода. Как говорится, «не баг, а фича», причём очень важная. Коэффициенты, стоящие при коллинеарных или высококоррелированных факторах, зануляются. Также чем выше коэффициент регуляризации, тем больше вероятность того, что коррелированные или малозначащие факторы будут исключены из модели. Чуть позже мы рассмотрим геометрическую интерпретацию и поймём, почему так происходит.
***
А пока давайте применим L1-регуляризацию к нашей полиномиальной модели третьей степени, прогнозирующей типичную цену на дома в районах Бостона.

Так как метод координатного спуска, который применяется для поиска коэффициентов, является численным, то необходима стандартизация исходных данных, чтобы обеспечить ему сходимость. Возьмём в качестве коэффициента регуляризации a = 0.01 и проверим качество полученной модели с помощью кросс-валидации по метрике MAPE:

In [39]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]

# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)

# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)

# создаём модель линейной регрессии c L1-регуляризацией
lasso = Lasso(alpha=0.1, max_iter=10000)

# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(lasso, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100))

MAPE на тренировочных фолдах: 12.44 %
MAPE на валидационных фолдах: 16.44 %


Видим, что с помощью L1-регуляризации удалось уменьшить ошибку модели (MAPE) на валидационных фолдах с 24.16% до 16.44% и сократить разницу в метриках на тренировочных и валидационных фолдах даже лучше, чем с этим справилась L2-регуляризация. Однако на самом деле мы просто удачно выбрали коэффициент регуляризации — при других значениях могли получиться совершенно другие результаты.

***
## **ELASTIC-NET**

Последний вид регуляризации (хотя их на самом деле больше), который мы рассмотрим, называется **Elastic-Net (эластичная сетка)**. Это комбинация L1- и L2-регуляризации.

Идея Elastic-Net состоит в том, что мы вводим ограничение как на норму весов порядка p = 1, так и на норму порядка p = 2. Тогда оптимизационная задача будет иметь вид:

![](https://lms.skillfactory.ru/assets/courseware/v1/7399b8f59ea951786a5825a891118f3f/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_15.png)

Немного модифицировав формулу функции Лагранжа, которая получается в результате такой задачи условной оптимизации, можно получить финальный результат:

![](data/83.PNG)

Аналитического решения у этой задачи нет, поэтому для её решения в sklearn, как и для модели Lasso, используется **координатный спуск**.

В sklearn эластичная сетка реализована в классе **ElasticNet** из пакета с линейными моделями — **linear_model**. За коэффициент alpha отвечает параметр *alpha*, за коэффициент lambda — *l1_ratio*.

Некоторые рекомендации от разработчиков ElasticNet:

* Использование параметра **l1_ratio** <0.01 приводит к нестабильным результатам.
* Вместо использования ElasticNet с **alpha**=0 лучше используйте **LinearRegression**, так как там применяется аналитическое решение, которое позволяет получать более точные решения, чем численный координатный спуск.

По традиции рассмотрим «игрушечный» пример работы с Elastic-Net, а затем применим эту модель к нашей задаче о домах в Бостоне.

In [40]:
from sklearn.linear_model import ElasticNet

### **Пример № 3**

Построить линейную регрессию с Elastic-Net-регуляризацией, если:

![](https://lms.skillfactory.ru/assets/courseware/v1/b5b24227906fb067bd68d190be99ee76/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_16.png)

Решить задачу с тремя комбинациями коэффициентов регуляризации:

![](data/84.PNG)

Составим матрицу наблюдений A для нашей задачи:

![](https://lms.skillfactory.ru/assets/courseware/v1/3cc67a89e412a9fe2ff9da81514c48b8/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_17.png)

Мы уже знаем, что матрица Грама  будет вырождена, а значит классического МНК-решения (не беря в расчёт сингулярное разложение) не получится.

Сразу переходим к построению регрессии с помощью ElasticNet.

In [41]:
# СЛУЧАЙ 1 a = 0.1 lambda = 0.2

# матрица наблюдений (включая столбец единиц)
A = np.array([
    [1, 1, 1, 1, 1],
    [1, 0, -3, 2, 4],
    [2, 0, -6, 4, 8]
]).T
# вектор целевого признака
y = np.array([4, 3, -4, 2, 7])
# получаем оценку коэффициентов регрессии 
lasso = ElasticNet(alpha=0.1, l1_ratio=0.2, fit_intercept=False)
lasso.fit(A, y)
print(lasso.coef_)

[1.13492457 0.19525842 0.6237965 ]


Обратим внимание, что зануления коэффициентов коллинеарных факторов x1 и x2 не произошло. Каждый из них вошёл в уравнение регрессии с ненулевым коэффициентом.



In [42]:
# СЛУЧАЙ 2 a = 0.1 lambda = 0.7

# получаем оценку коэффициентов регрессии
lasso = ElasticNet(alpha=0.1, l1_ratio=0.7, fit_intercept=False)
lasso.fit(A, y)
print(lasso.coef_)

[1.14379753 0.         0.71993025]


Обратим внимание, что произошло зануление коэффициентов. Это неспроста, так как мы понизили влияние L2-регуляризации и одновременно повысили влияние L1-регуляризации, которая, как мы уже знаем, приводит к исключению линейно зависимых факторов.

In [43]:
# СЛУЧАЙ 3 a = 0.1 lambda = 1

# получаем оценку коэффициентов регрессии
lasso = ElasticNet(alpha=0.1, l1_ratio=1, fit_intercept=False)
lasso.fit(A, y)
print(lasso.coef_)

[1.14925373 0.         0.71921642]


В округлениях значения не заметно, однако если присмотреться к коэффициентам более внимательно, можно увидеть, что мы получили в точности те же значения, которые получали для модели Lasso в примере № 2. Неудивительно, ведь мы обнулили влияние L2-регуляризации, выставив l1_ratio=1. По сути, мы использовали чистую модель Lasso.

Возникает вопрос: какой набор коэффициентов линейной регрессии всё-таки подходит лучше?

Ответить на него можно, только вычислив метрику качества и сравнив ошибки прогнозов каждой из полученных моделей. Мы уверены, вы можете сделать это самостоятельно.
***

Нам осталось только попробовать применить Elastic-Net к данным о недвижимости в Бостоне.

Как и для других моделей с регуляризацией, для Elastic-Net также лучше заранее позаботиться о стандартизации данных. В качестве коэффициентов регуляризации возьмём a = 0.1, lambda = 0.5. Качество модели проверим с помощью кросс-валидации на пяти фолдах, метрика — MAPE.

In [44]:
# выделяем интересующие нас факторы
X = boston_data[['LSTAT', 'PTRATIO', 'RM','CRIM']]
y = boston_data[['PRICE']]
# инициализируем стандартизатор StandardScaler
scaler = StandardScaler()
# подгоняем параметры стандартизатора (вычисляем среднее и СКО)
X = scaler.fit_transform(X)
# добавляем полиномиальные признаки
poly = PolynomialFeatures(degree=3, include_bias=False)
X = poly.fit_transform(X)
# создаём модель линейной регрессии c L1- и L2-регуляризациями
lasso = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)
# оцениваем качество модели на кросс-валидации
cv_results = cross_validate(lasso, X, y, scoring='neg_mean_absolute_percentage_error', cv=5, return_train_score=True)
print('MAPE на тренировочных фолдах: {:.2f} %'.format(-cv_results['train_score'].mean()* 100))
print('MAPE на валидационных фолдах: {:.2f} %'.format(-cv_results['test_score'].mean() * 100)) 

MAPE на тренировочных фолдах: 12.65 %
MAPE на валидационных фолдах: 15.70 %


Итак, Elastic-Net позволил нам уменьшить значение MAPE на валидационных фолдах с 24.16% до 15.7%. Отличный результат! Он получился лучше, чем у моделей Ridge и Lasso, но опять же скажем, что так бывает не всегда.

→ На практике при использовании моделей с регуляризацией стоит подбирать значения коэффициентов регуляризации с помощью методов подбора гиперпараметров, которые мы изучали в модуле «ML-7. Оптимизация гиперпараметров модели». Только после подбора гиперпараметров можно сделать вывод, какая из моделей показывает наилучшие результаты для решения конкретной задачи. Надеемся, вы помните, как подбираются гиперпараметры (если нет, освежите знания в модуле [ML-7](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/jump_to_id/cfba32dd83a948ec811afc5132f079c5)).

***
## ***ГЕОМЕТРИЧЕСКАЯ ИНТЕРПРЕТАЦИЯ РЕГУЛЯРИЗАЦИИ**

Напоследок поговорим о геометрической интерпретации регуляризации. В дальнейшем, изучая теорию оптимизации, мы увидим, что задача условной оптимизации

![](https://lms.skillfactory.ru/assets/courseware/v1/ff4221dc97f0998ee63d55765b87f02d/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_18.png)

![](data/85.PNG)

![](https://lms.skillfactory.ru/assets/courseware/v1/d584cd8689e7909149674a3805917c48/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/MATHML_md2_7_19.png)

Концентрическими кругами обозначены линии равного уровня функции L(w). Для каждого конкретного набора данных она будет иметь разный вид, но смысл будет тем же. Голубой областью обозначены ромб и окружность, которые задаёт L1- и L2-норма вектора весов соответственно.

* Если бы мы использовали классическую линейную регрессию, то МНК приводил бы нас в точку истинного минимума функции L(w) — в центр, из которого исходят концентрические круги. Это была бы некоторая комбинация параметров w0 и w1.
* В случае, когда мы используем модель линейной регрессии с регуляризацией, мы будем пытаться найти такую комбинацию w0 и w1, которая доставляет минимум функции L(w), но при этом не выходит за границы ромба (или окружности). Таким образом, вместо истинного минимума мы находим так называемый **псевдоминимум**.

Заметим, что у ромба вероятность коснуться концентрического круга одной из своих вершин больше, чем у окружности — своей верхней/нижней/правой/левой точкой. Точка касания в вершине ромба — это точка, в которой либо w0 = 0 либо w1 = 0. То есть L1-регуляризация склонна с большей вероятностью занулять коэффициенты линейной регрессии, чем L2-регуляризация.

Величина диагонали ромба и радиуса окружности зависят от величины коэффициента регуляризации ***a***: чем больше ***a***, тем меньше ромб/окружность, а значит тем дальше псевдоминимум будет находиться от истинного минимума, и наоборот.